In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.spatial import distance
import xml.etree.ElementTree as ET
from shapely.geometry import Polygon, LineString
import pandas as pd
#from src.generate_traffic_data import clean_carla, spawn_scene, run_scene
import carla
import os
os.getcwd()
from datetime import datetime
import time
import random
from tqdm import tqdm
import networkx as nx
import numpy as np
import pickle

os.chdir('../..')
os.getcwd()

#from graph_creator.MapGraph import MapGraph
#from graph_creator.ActorGraph import ActorGraph

from graph_creator.plot_graphs import plot_lane_map_advanced, add_actors_to_map, add_actor_edges_to_map

### Start comparing graphs:

In [3]:
from glob import glob
import pickle
import networkx as nx
import random
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

graph_paths = glob("/home/tmuehlen/repos/graph_coverage/actor_graphs/carla/*.pkl")
len(graph_paths)

3993

In [4]:
random.sample(list(range(len(graph_paths))), 10)

[270, 157, 1452, 376, 959, 1480, 3443, 2266, 3287, 969]

In [ ]:
# Define a random set of coverage situations with 3 nodes each
# This still needs to be replaced by more strategically selected graphs

#idxs = [100, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000]
idxs = random.sample(list(range(len(graph_paths))), 10)

coverage_graphs = {}
cov_data = {}

for idx in idxs:
    graph_path = graph_paths[0]
    with open(graph_path, "rb") as file:
        ag_nx = pickle.load(file)
    #random.sample(ag_nx.nodes, 3)
    coverage_graphs[idx] = ag_nx.subgraph(random.sample(list(ag_nx.nodes), 3))
    cov_data[idx] = []

cov_data["degree"] = []
cov_data["density"] = []
cov_data["diameter"] = []


In [8]:
ag_nx.nodes(data=True)

NodeDataView({'615': {'lane_id': '24_1', 'lane_ids': ['24_1'], 's': 0.2500001788139343, 'xyz': <POINT Z (-28.505 -64.788 -0.001)>, 'lon_speed': 4.256774791250496, 'actor_type': <ActorType.VEHICLE: 'VEHICLE'>, 'lane_change': False}, '630': {'lane_id': '7_-1', 'lane_ids': ['7_-1'], 's': 8.250000700354576, 'xyz': <POINT Z (-19.196 -61.195 0.0003)>, 'lon_speed': 4.968893154799291, 'actor_type': <ActorType.VEHICLE: 'VEHICLE'>, 'lane_change': False}, '624': {'lane_id': '7_-1', 'lane_ids': ['7_-1'], 's': 20.999999836087227, 'xyz': <POINT Z (-6.375 -61.024 0.0009)>, 'lon_speed': 5.056757849845666, 'actor_type': <ActorType.VEHICLE: 'VEHICLE'>, 'lane_change': False}})

In [ ]:
for graph_path in graph_paths:
    with open(graph_path, "rb") as file:
        ag_nx = pickle.load(file)

    for key in coverage_graphs.keys():
        GM = nx.algorithms.isomorphism.DiGraphMatcher(
            ag_nx, coverage_graphs[key],
            node_match=nx.algorithms.isomorphism.categorical_node_match('actor_type', None),
            # node_match=nx.algorithms.isomorphism.categorical_node_match(['actor_type', 'lane_change'], [None, None]),
            edge_match=nx.algorithms.isomorphism.categorical_edge_match('edge_type', None)
        )
        cov_data[key].append(GM.subgraph_is_isomorphic())
    
    cov_data["degree"].append(sum(dict(ag_nx.degree()).values()) / len(ag_nx.nodes()))
    cov_data["density"].append(nx.density(ag_nx))
    cov_data["diameter"].append(nx.diameter(ag_nx))

cov_data_df = pd.DataFrame(cov_data)
cov_data_df.mean()

In [ ]:
list(cov_data_df.columns[:10])


In [ ]:
cov_data_df.groupby(list(cov_data_df.columns[:11]), as_index=False).size()

In [ ]:
# sns.pairplot(data=cov_data_df)
# plt.show()


# Notebook junkyard